# ハードウェア確認

In [ ]:
from psutil import cpu_count, virtual_memory

# CPU情報の取得
print(f'CPU count: {cpu_count()}')

# メモリ情報の取得
mem = virtual_memory()
print(f'Memory: {mem.total / 1024**3:.2f} GB')

# ディスク情報の取得
!df -h / | grep -E "^/"

# GPU情報の取得
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs: {gpu_devices}')

CPU count: 2
Memory: 12.67 GB
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# ベンチマークテスト

In [ ]:
!pip install pulp
!pip install ortools

## PuLPソルバー

In [ ]:
import time
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import numpy as np

# 問題の規模を定義
num_variables = 1000
num_constraints = 500

# 問題を定義
model = LpProblem(name="large-scale-problem", sense=LpMaximize)

# 変数を定義
variables = [LpVariable(f"x{i}", lowBound=0) for i in range(num_variables)]

# 目的関数を追加（ランダムな係数を使用）
objective_coeffs = np.random.rand(num_variables)
model += lpSum([objective_coeffs[i] * variables[i] for i in range(num_variables)])

# 制約を追加（ランダムな係数と定数を使用）
for _ in range(num_constraints):
    constraint_coeffs = np.random.rand(num_variables)
    rhs = np.random.rand() * 1000
    model += lpSum([constraint_coeffs[i] * variables[i] for i in range(num_variables)]) <= rhs

# 問題を解く
start_time = time.time()
status = model.solve()
end_time = time.time()

print(f'Solution status: {status}')
print(f'Elapsed time: {end_time - start_time:.2f} seconds')


Solution status: 1
Elapsed time: 1.44 seconds


## OR-Toolsソルバー

In [ ]:
import time
import numpy as np
from ortools.linear_solver import pywraplp

# 問題の規模を定義
num_variables = 1000
num_constraints = 500

# OR-Toolsソルバーの作成
solver = pywraplp.Solver.CreateSolver('GLOP')

if not solver:
    raise Exception("Solver not created.")

# 変数を定義
variables = []
for i in range(num_variables):
    variables.append(solver.NumVar(0, solver.infinity(), f'x{i}'))

# 目的関数を追加（ランダムな係数を使用）
objective = solver.Objective()
objective_coeffs = np.random.rand(num_variables)
for i in range(num_variables):
    objective.SetCoefficient(variables[i], objective_coeffs[i])
objective.SetMaximization()

# 制約を追加（ランダムな係数と定数を使用）
for _ in range(num_constraints):
    constraint_coeffs = np.random.rand(num_variables)
    rhs = np.random.rand() * 1000
    constraint = solver.RowConstraint(-solver.infinity(), rhs, '')
    for i in range(num_variables):
        constraint.SetCoefficient(variables[i], constraint_coeffs[i])

# 問題を解く
start_time = time.time()
status = solver.Solve()
end_time = time.time()

print(f'Solution status: {status}')
print(f'Elapsed time with OR-Tools: {end_time - start_time:.2f} seconds')


Solution status: 0
Elapsed time with OR-Tools: 0.10 seconds


# ベンチマークテスト(行列演算)

## ベンチマークスクリプト

In [ ]:
import numpy as np
import time

# 行列のサイズを定義
matrix_size = 5000

# ランダムな行列を生成
A = np.random.rand(matrix_size, matrix_size)
B = np.random.rand(matrix_size, matrix_size)

# 行列の乗算を実行し、時間を計測
start_time = time.time()
C = np.dot(A, B)
end_time = time.time()

print(f'Elapsed time for matrix multiplication of size {matrix_size}x{matrix_size}: {end_time - start_time:.2f} seconds')


Elapsed time for matrix multiplication of size 5000x5000: 4.07 seconds


## GPUを使用した行列演算

In [ ]:
import tensorflow as tf

# TensorFlowのバージョン確認
print(f'TensorFlow version: {tf.__version__}')

# ランダムな行列を生成
A = tf.random.uniform((matrix_size, matrix_size))
B = tf.random.uniform((matrix_size, matrix_size))

# GPUを使用して行列の乗算を実行し、時間を計測
start_time = time.time()
C = tf.matmul(A, B)
tf_end_time = time.time()

print(f'Elapsed time for matrix multiplication with TensorFlow and GPU: {tf_end_time - start_time:.2f} seconds')

TensorFlow version: 2.15.0
Elapsed time for matrix multiplication with TensorFlow and GPU: 0.15 seconds
